In [18]:
import numpy as np
import json
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation  

In [19]:
##Rotation Euler
def roationconvert_nt(phi,theta,psi,data):
    cphi = np.cos(phi)
    sphi = np.sin(phi)
    ctheta = np.cos(theta)
    stheta = np.sin(theta)
    cpsi = np.cos(psi)
    spsi = np.sin(psi)
    R = np.array([
        [cpsi * ctheta, -spsi * ctheta + cpsi * stheta * sphi, spsi * sphi + cpsi * stheta * cphi],
        [spsi * ctheta,  cpsi * ctheta + spsi * stheta * sphi, -cpsi * sphi + spsi * stheta * cphi],
        [-stheta,        ctheta * sphi,                         ctheta * cphi]
    ])
    # Rt = np.transpose(R)
    # Perform matrix multiplication (rotation)
    rotated_data = R @ data
    
    return rotated_data
def roationconvert(phi,theta,psi,data):
    cphi = np.cos(phi)
    sphi = np.sin(phi)
    ctheta = np.cos(theta)
    stheta = np.sin(theta)
    cpsi = np.cos(psi)
    spsi = np.sin(psi)
    R = np.array([
        [cpsi * ctheta, -spsi * ctheta + cpsi * stheta * sphi, spsi * sphi + cpsi * stheta * cphi],
        [spsi * ctheta,  cpsi * ctheta + spsi * stheta * sphi, -cpsi * sphi + spsi * stheta * cphi],
        [-stheta,        ctheta * sphi,                         ctheta * cphi]
    ])
    Rt = np.transpose(R)
    # Perform matrix multiplication (rotation)
    rotated_data = Rt @ data
    
    return rotated_data


In [20]:
    # GetOrignal Data
with open('/media/adminnio/Volume/jasjappan/blender/orginalposes/transforms_val.json', 'r') as file:
     transform_data = json.load(file)

poses = np.array([frame['transform_matrix'] for frame in transform_data['frames']])

origins = poses[:,:-1,-1]
rotation_matrix = poses[:,:-1,0:3]

#Extract position and translation matrix from poses

r =  Rotation.from_matrix(rotation_matrix)
angles = r.as_euler("zyx",degrees=True)
x = origins[:,0]
y = origins[:,1]
z = origins[:,2]
phi = angles[:,0]
theta = angles[:,1]
psi = angles[:,2]

In [21]:
#Simulate IMU

dt = 1
xdot = np.gradient(x, dt)
ydot = np.gradient(y,dt)
zdot = np.gradient(z,dt)
phidot = np.gradient(phi, dt)
thetadot = np.gradient(theta, dt)
psidot = np.gradient(psi, dt)
posdot=np.array([xdot,ydot,zdot])
rotdot = np.array([phidot,thetadot,psidot])
linearvelocity = []
angularvelocity = []
i = 0
for pos in posdot.T:  
    linearvelocity.append(roationconvert(phi[i], theta[i], psi[i], pos))
    i += 1

i = 0
for rot in rotdot.T: 
    angularvelocity.append(roationconvert(phi[i], theta[i], psi[i], rot))
    i += 1

linearvelocity = np.array(linearvelocity)
angularvelocity = np.array(angularvelocity)
u = linearvelocity[:,0]
v = linearvelocity[:,1]
w = linearvelocity[:,2]

p = angularvelocity[:,0]
q = angularvelocity[:,1]
r = angularvelocity[:,2]

xdotdot = np.gradient(u, dt)
ydotdot = np.gradient(v,dt)
zdotdot = np.gradient(w,dt)

linearacc = np.array([xdotdot,ydotdot,zdotdot])
angularvel = np.array([p,q,r])
#accelerometre data & gyroscope data

In [22]:
time =np.arange(start=0, stop=170)
linearacc = np.transpose(linearacc) #Shape is now 170,3
angularvel = np.transpose(angularvel) #Shape is now 170,3

print(np.shape(angularvel))


(170, 3)


In [26]:
thetaacc = []
phihatacc = np.arctan(linearacc[:,1],linearacc[:,2])
g = np.full(170, fill_value = 9.8)
thetaacc.append(np.arcsin(linearacc[:,0],g))
eulerestimatephi = []
thetaacc = np.array(thetaacc)
i = 0
# np.transpose(thetaacc)
for rot in angularvel: 
    eulerestimatephi.append(roationconvert_nt(phihatacc[i], thetaacc[0,i], psi[i], rot))
    i += 1

eulerestimatephi = np.array(eulerestimatephi)

In [38]:
dt =1
xhat = []
k1 = 0.9
k2 = 0.1
ex = 0
b = 0
xk = 0
for i in range(170):
    xk = (k1*ex + eulerestimatephi[i] + b) * dt + xk
    ex = xk - phi[i]
    b = b + (k2*ex)*dt
    xhat.append(xk)
    # return xhat

In [39]:
eulerestimatephi = np.array(eulerestimatephi)
eulerestimatephi[0,:]

array([-0.01802377, -0.10818666, -0.05056257])

In [40]:
xhat

[array([-0.01802377, -0.10818666, -0.05056257]),
 array([-4.96990913, -5.27785554, -5.09411681]),
 array([-15.32199049, -16.15182621, -15.70016312]),
 array([-36.91804986, -38.90177868, -37.92268559]),
 array([-81.98186897, -86.41381559, -84.41688643]),
 array([-176.10145037, -185.62242702, -181.64281827]),
 array([-372.84648798, -392.87126902, -384.94267186]),
 array([-784.33645588, -826.0668111 , -810.09328205]),
 array([-1645.14098003, -1731.94090763, -1699.30695507]),
 array([-3445.87080878, -3626.70365365, -3559.24112332]),
 array([-7212.54673573, -7590.13560425, -7449.58235531]),
 array([-15090.96221144, -15880.5819614 , -15586.61636175]),
 array([-31569.12073806, -33221.24712489, -32605.74336642]),
 array([-66034.34410057, -69490.8307417 , -68202.36033742]),
 array([-138121.6519238 , -145351.64132096, -142655.82255947]),
 array([-288899.96867021, -304021.62420124, -298383.33675491]),
 array([-604269.12964047, -635896.78862482, -624105.13866635]),
 array([-1263897.66514992, -1330

In [34]:
phi

array([ 4.92296501e+00,  4.90472447e+00,  4.87450041e+00,  4.83244464e+00,
        4.77868365e+00,  4.71335849e+00,  4.63659937e+00,  4.54852724e+00,
        4.44925211e+00,  4.33886359e+00,  4.21743809e+00,  4.08502682e+00,
        3.94165590e+00,  3.78731828e+00,  3.62162452e+00,  3.44209523e+00,
        3.25112200e+00,  3.04851014e+00,  2.83399232e+00,  2.60723782e+00,
        2.36783257e+00,  2.11470062e+00,  1.84760067e+00,  1.56593012e+00,
        1.26881659e+00,  9.55208720e-01,  6.24959622e-01,  2.75810019e-01,
       -9.40657164e-02, -4.86641208e-01, -9.02730050e-01, -1.34558989e+00,
       -1.81916609e+00, -2.32725936e+00, -2.87260707e+00, -3.46195277e+00,
       -4.10206005e+00, -4.80087755e+00, -5.56816888e+00, -6.41609541e+00,
       -7.36043317e+00, -8.42337994e+00, -9.62730069e+00, -1.10054594e+01,
       -1.26094669e+01, -1.44923726e+01, -1.67371001e+01, -1.94684513e+01,
       -2.28467521e+01, -2.71129693e+01, -3.26107122e+01, -3.98530445e+01,
       -4.94253436e+01, -